# Business Problem:

'ABC' flower company is finding difficult to classify flowers that are grown in their garden based on their:
  1. sepal_length_cm
  2. sepal_width_cm
  3. petal_length_cm
  4. petal_width_cm
  
  
# Information from domain experts

1. There will be ouliers in the dataset.
2. Flowers names are not populated properly all the time.
3. Some data are in unit 'm' while the standard unit is 'cm'.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
import seaborn as sns
from pandas.tools.plotting import scatter_matrix

Reference: https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html

In [ ]:
#import dataset
data_df = pd.

# 1. Exploring the data structure

## 1.1 Take a look at the dataset

View top 5 rows

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.head.html

## 1.2 Get description of data

Use 'info' to get description of data

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.info.html

From the above observation, you can be observe that,
1. There are 150 total entries. Each data point has 4 features and 1 class which is to be predicted.
2. Out of 150 total entries, there are 5 entries in petal_width_cm whose value is not known.

## 1.3 Summary of data

Use pandas describe to get summary

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.describe.html

Summary of the numerical values. Summary includes obtaining mean, standard deviation, minimum and maximim value.

# 2. Data Cleaning

## 2.1  Drop datapoints with 'NA' 

Use 'dropna' from pandas to drop dataset with 'NA'. Try using 'subset' parameter to remove 'NA' values from petal_width_cm

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html
    
    

In [ ]:
#To view if the datapoints with 'NA' has been removed
(data_frame).info()

## 2.2 Explore categorical data

Try pandas 'value_counts' to see if there is any misspelled category name.

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.value_counts.html

#### Hint: Can you spot the error????

1. For 5 data points 'Iris-versicolor' has been specified as 'versicolor' 
2. For 1 data points, 'Iris-setosa' has been specified as 'Iris-setossa'

Use replace in pandas to correct misspelled words. Try using 'inplace' parameter to make the changes permanent.

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.replace.html

In [ ]:

#.value_counts()

## 2.3 Outlier detection

use 'hist' in pandas to observe any outliers in the features. Play around with 'bins' parameter to spot the outlier.

Hint: Outlier is there in 'sepal...' feature

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.hist.html

Use 'drop' in pandas to remove rows that contain outlier

https://thispointer.com/python-pandas-how-to-drop-rows-in-dataframe-by-conditions-on-column-values/

In [5]:
#use hist pandas to view if it has been removed.

## 2.4 Converting datapoints from unit 'm' to 'cm'

Few values (which are less than 1) in 'sepal_length_cm' are in 'm' try converting them into 'cm'.

1. Explore 'loc' 
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.loc.html

Syntax:
    dataframe.loc[dataframe.feature_name < value, ['feature_name']] = dataframe['feature_name']*100
    Here,
    * dataframe will be the name of the dataframe that you are using.
    * replace feature name with 'sepal_length_cm'.
    * value will be 1(confirm using plot if it is 1).

Finally, use 'hist' to confirm if it has been changed.

## 2.5 Handling categorical variable

Explore deep copy and try copying the dataframe to other.
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.copy.html

Try one-hot encoding the 'class' column (feature). One other easy way is to use 'get_dummies'.

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.get_dummies.html

# 3. Visualization

## 3.1 Get correlation between features

Get correlation values using pandas corr

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.corr.html

In [ ]:
#Set figure size
plt.rcParams["figure.figsize"] = [15,10]

Use seaborn heatmap to visualize the correlation matrix.

## 3.2  Scatter Plot
It is plotted between each feature against the other. Distribution of the data is can be clearly obtained using the scatter plot. It can be inferred from the scatter plot that data can be linearly seperated.

https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.plotting.scatter_matrix.html